In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

ModuleNotFoundError: No module named 'torch._C'

In [3]:
class CheckpointHandler:
    def __init__(self, best_valid_loss=float('inf')):
        self.best_valid_loss = best_valid_loss
    
    def save_best_model(self, state, filename):
        if state['loss'] < self.best_valid_loss:
            self.best_valid_loss = state['loss']
            torch.save(state, filename)
            print('checkpoint file updated')

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Conv2d(1, 32, kernel_size= 3),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2, 2), stride= 2),
                                 nn.Conv2d(32, 64, kernel_size= 3),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2, 2), stride= 2),
                                 nn.Conv2d(64, 32, kernel_size= 3),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2, 2), stride= 2)
                                 )
        self.classify_head = nn.Sequential(nn.Flatten(),
                                           nn.Linear(32, 20, bias= True),
                                           nn.Linear(20, 10, bias= True))
    def forward(self, x):
        return self.classify_head(self.net(x))

In [ ]:
model = CNN()
model.to('cuda')
optimizer = optim.SGD(model.parameters(), lr= 0.001)
criterion = nn.CrossEntropyLoss()
checkpoint_handler = CheckpointHandler()

In [ ]:
transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, ), (0.5, ))])
train = datasets.MNIST('.', train= True, download= True, transform= transforms)
test = datasets.MNIST('.', download= True, train= False, transform= transforms)
train_loader = DataLoader(train, batch_size= 64, shuffle= True)
test_loader = DataLoader(test, batch_size= 64)

In [ ]:
for epoch in range(10):
    model.train()
    running_loss = 0
    for input, target in train_loader:
        input, target = input.to('cuda'), target.to('cuda')
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch - {epoch}, loss = {running_loss}')

    state = {'epoch': epoch,
             'model_state_dict': model.state_dict(),
             'optimizer_state_dict': optimizer.state_dict(),
             'loss': running_loss}

    checkpoint_handler.save_best_model(state, './q3_checkpoints/checkpoint_best.pth')

In [ ]:
# retraining
model = CNN()
checkpoint = torch.load('./q3_checkpoints/checkpoint_best.pth', weights_only= False)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
criterion = nn.CrossEntropyLoss()

In [ ]:
model.to('cuda')
for epoch in range(10):
    model.train()
    running_loss = 0
    for input, target in train_loader:
        input, target = input.to('cuda'), target.to('cuda')
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch - {epoch}, loss = {running_loss}')

    state = {'epoch': epoch,
             'model_state_dict': model.state_dict(),
             'optimizer_state_dict': optimizer.state_dict(),
             'loss': running_loss}

    checkpoint_handler.save_best_model(state, './q3_checkpoints/checkpoint_best.pth')

In [ ]:
all_preds, all_target = [], []
model.eval()
with torch.no_grad():
    for input, target in test_loader:
        input, target = input.to('cuda'), target.to('cuda')
        output = model(input)
        val, index = torch.max(output, dim= 1)
        all_preds.extend(index.to('cpu'))
        all_target.extend(target.to('cpu'))
from sklearn.metrics import accuracy_score
print(accuracy_score(all_preds, all_target))